<a href="https://colab.research.google.com/github/PradyumnaKrishna/Colab-Hacks/blob/master/Colab%20SSH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Colab SSH** : SSH to Colab Instance

There are Two methods of doing same thing :
 - Using colab_ssh pip library
 - Manually configuring SSHD

Run command on your Local Computer to connect using SSH

    ssh root@0.tcp.ngrok.io -p <givenport>

And then enter the password

<br>

> **Warning :** This notebook is against the Policy of Colab. Use it on your own risk



# Using **colab_ssh** Library

### Installing colab_ssh Library

In [ ]:
!pip install colab_ssh --upgrade

### Generate Root Password


In [ ]:
import random, string
password = ''.join(random.choice(string.ascii_letters + string.digits) for i in range(20))

### Request ngrok Token

In [ ]:
print("Copy authtoken from https://dashboard.ngrok.com/auth")
ngrokToken = input("Enter Token : ")

### Launch SSH

In [ ]:
from colab_ssh import launch_ssh, init_git
launch_ssh(ngrokToken, password)

### Print Details

In [ ]:
# Get Public Address
print("Run command : ssh root@", end='')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'][6:].replace(':', ' -p '))"

# Print Password
print("Password : ", password)

# Manually configuring SSHD

### Generate root password


In [ ]:
import random, string
password = ''.join(random.choice(string.ascii_letters + string.digits) for i in range(20))

### Download ngrok

In [ ]:
!wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -qq -n ngrok-stable-linux-amd64.zip

### Setup sshd

In [ ]:
!apt-get install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen > /dev/null

### Set root password

In [ ]:
! echo root:$password | chpasswd
! mkdir -p /var/run/sshd
! echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
! echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config
! echo "LD_LIBRARY_PATH=/usr/lib64-nvidia" >> /root/.bashrc
! echo "export LD_LIBRARY_PATH" >> /root/.bashrc

### Run sshd

In [ ]:
get_ipython().system_raw('/usr/sbin/sshd -D &')

### Ask token

In [ ]:
print("Copy authtoken from https://dashboard.ngrok.com/auth")
import getpass
authtoken = getpass.getpass()

### Create tunnel

In [ ]:
get_ipython().system_raw('./ngrok authtoken $authtoken && ./ngrok tcp 22 &')

### Print root password

In [ ]:
print("Root password: {}".format(password))

### Get public address

In [ ]:
# Get Public Address
print("Run command : ssh root@", end='')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'][6:].replace(':', ' -p '))"

# Print Password
print("Password : ", password)